## 2유형 [2] Penguins

In [22]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import numpy as np
import pandas as pd
X_test = pd.read_csv("./datasets/Part2/penguin_X_test.csv")
X_train = pd.read_csv("./datasets/Part2/penguin_X_train.csv")
y_train = pd.read_csv("./datasets/Part2/penguin_y_train.csv")

# 사용자 코딩

### 1. 데이터 전처리
# 1) info 함수로 결측치 확인
# print(X_train.info())

# 2) 결측치를 제거하는 방식을 사용하려면 y_train과 갯수를 맞춰주어야 함 => concat으로 합치고 제거
train = pd.concat([X_train, y_train], axis=1)
# print(train.loc[(train.sex.isna())|train.bill_length_mm.isna()|train.bill_depth_mm.isna()|train.flipper_length_mm.isna()])

train = train.dropna()
train.reset_index(drop=True, inplace=True)

# 3) 다시 독립변수&종속변수로 분리 후, 기술통계량 확인
X_train = train[['species', 'island', 'sex', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y_train = train[['body_mass_g']]

# print(X_train.describe())

# 컬럼 저장
COL_DEL = []
COL_NUM = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']
COL_CAT = ['species', 'island', 'sex']
COL_Y = ['body_mass_g']

# 4) 범주형 독립변수에 대한 레이블 인코딩 (원-핫 인코딩)
X = pd.concat([X_train, X_test])

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(X[COL_CAT])

X_train_res = ohe.transform(X_train[COL_CAT])
X_test_res = ohe.transform(X_test[COL_CAT])

# print(X_train_res)

# 5) 레이블 인코딩 후 데이터프레임으로 변환
X_train_ohe = pd.DataFrame(X_train_res.todense(), columns=ohe.get_feature_names())
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns=ohe.get_feature_names())
# print(X_train_ohe)

X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis=1)
X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis=1)

# -------------------------

### 2. 모델 구축
# 1) 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train_fin, y_train, test_size=0.3)

# 2) 데이터 스케일링 (MinMaxScaler)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test_fin[COL_NUM] = scaler.transform(X_test_fin[COL_NUM])

# 3) 회귀 모델 구축
from sklearn.linear_model import LinearRegression

modelLR = LinearRegression()
modelLR.fit(X_tr, y_tr)

y_val_pred = modelLR.predict(X_val)
# print(y_val_pred)

# 4) 회귀 절편과 기울기 구하기
# print(modelLR.intercept_)

coef = pd.Series(data=modelLR.coef_[0], index=X_train_fin.columns)
# print(coef.sort_values())

# -------------------------

### 3. 모델 평가 (RMSE)
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_val, y_val_pred)
rmse = mean_squared_error(y_val, y_val_pred, squared=False)

# print('MSE : {0:.3f}, RMSE : {1:.3f}'.format(mse, rmse))

# -------------------------

### 4. 제출 파일 저장
# 1) 문제에서 요구하는 제출물 그대로 predict 함수 적용
y_pred = modelLR.predict(X_test_fin)
# print(y_pred)

# 2) 제출 파일 형태로 저장
pd.DataFrame({'body_mass_g': y_pred[:,0]}).to_csv('./res/004000000.csv', index=False)

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)


C:\Users\phi49\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\phi49\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
